### 선형 회귀 분석이란?
**피처 데이터와 타깃 데이터 간의 선형 관계를 파악하는 알고리즘**

### 보스턴 집값 데이터를 예측하는 모형 제작
**선형 회귀 분석, 릿지 회귀 분석, 라쏘 회귀 분석, 엘라스틱 넷 사용**

In [5]:
import pandas as pd
import numpy as np
# 데이터 불러오기
# 보스턴 집값 데이터 (인종 차별적인 요소로 scikit-learn에서 제거, url 로드)
data_url = "http://lib.stat.cmu.edu/datasets/boston"

# 데이터 로드 (원시 문자열을 사용하여 '\s+'가 정규 표현식으로 인식되도록 함)
raw_df = pd.read_csv(data_url, sep=r"\s+", skiprows=22, header=None)

# 데이터와 타겟을 분리
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# 피처, 타깃 데이터 지정
X = pd.DataFrame(data)
y = pd.DataFrame(target) 

In [7]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te=train_test_split(X, y, random_state=0)

In [8]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [10]:
# 데이터 학습(선형 회귀 분석)
from sklearn.linear_model import LinearRegression
clf_lr = LinearRegression()
clf_lr.fit(X_tn_std, y_tn)

LinearRegression()

In [13]:
# 선형 회귀 분석 계수, 상수항 확인

# 추정 회귀 계수
print(clf_lr.coef_)

# 추정 상수항(y 절편)
print(clf_lr.intercept_)

[[-0.97100092  1.04667838 -0.04044753  0.59408776 -1.80876877  2.60991991
  -0.19823317 -3.00216551  2.08021582 -1.93289037 -2.15743759  0.75199122
  -3.59027047]]
[22.60870712]


In [14]:
# 데이터 학습(L2 제약식 적용, 릿지 회귀 분석)
from sklearn.linear_model import Ridge

# alpha : 값이 클수록 강한 제약식(기본값 : 1)
clf_ridge = Ridge(alpha=1)
clf_ridge.fit(X_tn_std, y_tn)

Ridge(alpha=1)

In [15]:
# 릿지 회귀 분석 계수, 상수항 확인

# 추정 회귀 계수 
print(clf_ridge.coef_)

# 추정 상수항
print(clf_ridge.intercept_)

[[-0.96187481  1.02775462 -0.06861144  0.59814087 -1.77318401  2.6205672
  -0.20466821 -2.96504904  2.00091047 -1.85840697 -2.14955893  0.75175979
  -3.57350065]]
[22.60870712]


In [17]:
# 데이터 학습(L1 제약식 적용, 라쏘 회귀 분석)
from sklearn.linear_model import Lasso

# alpha : 제약의 정도(1을 디폴트)
clf_lasso = Lasso(alpha=0.01)
clf_lasso.fit(X_tn_std, y_tn)

Lasso(alpha=0.01)

In [18]:
#라쏘 회귀 분석 계수, 상수항 확인

# 추정 회귀 계수
print(clf_lasso.coef_)

# 추정 상수항
print(clf_lasso.intercept_)

[-0.93949205  1.01037722 -0.05747479  0.59232437 -1.76160385  2.62290366
 -0.17911018 -2.92328686  1.93398258 -1.81118512 -2.14705184  0.73964238
 -3.59732302]
[22.60870712]


In [20]:
# 데이터 학습(엘라스틱 넷)
from sklearn.linear_model import ElasticNet

# alpha : L1 제약식의 크기와 L2 제약식의 크기의 합(전체 제약식의 크기)
# l1_ratio : 전체 제약의 크기인 alpha 값에서 L1 제약이 차지하는 비율을 의미(0과 1 사잇값을 가진다)
clf_elastic = ElasticNet(alpha=0.01, l1_ratio=0.01)
clf_elastic.fit(X_tn_std, y_tn)

ElasticNet(alpha=0.01, l1_ratio=0.01)

In [21]:
# 엘라스틱 넷 계수, 상수항 확인

# 추정 회귀 계수
print(clf_elastic.coef_)

# 추정 상수항
print(clf_elastic.intercept_)

[-0.93905353  0.98057259 -0.13487248  0.60767135 -1.68277217  2.64642801
 -0.21978011 -2.86727682  1.80930385 -1.68204979 -2.12935659  0.75122826
 -3.52939748]
[22.60870712]


In [22]:
# 데이터 예측
pred_lr = clf_lr.predict(X_te_std)
pred_ridge = clf_ridge.predict(X_te_std)
pred_lasso = clf_lasso.predict(X_te_std)
pred_elastic = clf_elastic.predict(X_te_std)

In [24]:
# 모형 평가 
# R 제곱값(전체 모형에서 설명 가능한 분산의 비율, 1에 가까울수록 높은 성능)
from sklearn.metrics import r2_score 

# 선형 회귀 분석
print(r2_score(y_te, pred_lr))

# 릿지 회귀 분석
print(r2_score(y_te, pred_ridge))

# 라쏘 회귀 분석
print(r2_score(y_te, pred_lasso))

# 엘라스틱 넷 회귀 분석
print(r2_score(y_te, pred_elastic))

0.6354638433202133
0.6345884564889055
0.6343061000666703
0.6322273400977834


In [25]:
# 모형 평가 - MSE(오차 제곱합, 작을수록 성능이 좋음)
from sklearn.metrics import mean_squared_error

# 선형 회귀 분석
print(mean_squared_error(y_te, pred_lr))

# 릿지 회귀 분석
print(mean_squared_error(y_te, pred_ridge))

# 라쏘 회귀 분석
print(mean_squared_error(y_te, pred_lasso))

# 엘라스틱 넷 회귀 분석
print(mean_squared_error(y_te, pred_elastic))

29.782245092302325
29.853763334547597
29.87683157624681
30.046664219036877
